### Routing

Routing is a pattern that directs different types of requests to specialized handlers. This allows for optimized processing of distinct request types while maintaining a clean separation of concerns.

#### Calendar Assistant Example

Our calendar assistant demonstrates routing between new event creation and event modification:

![Alt Text](./images/Routing.png)


In [13]:
from typing import Optional, Literal
from pydantic import BaseModel, Field
from datetime import datetime
from openai import OpenAI
import os
import logging

In [14]:
# Set up logging configuration
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)
logger = logging.getLogger(__name__)

In [15]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
model = "gpt-4o"

#### Specialized Handlers

- New Event Handler: Creates calendar events
- Modify Event Handler: Updates existing events
- Each optimized for its specific task

##### Step 1: Define the data models for routing and responses


In [16]:
class CalendarRequestType(BaseModel):
    """Router LLM call: Determine the type of calendar request"""

    request_type: Literal["new_event", "modify_event", "other"] = Field(
        description="Type of calendar request being made"
    )
    confidence_score: float = Field(description="Confidence score between 0 and 1")
    description: str = Field(description="Cleaned description of the request")

In [17]:
class NewEventDetails(BaseModel):
    """Details for creating a new event"""

    name: str = Field(description="Name of the event")
    date: str = Field(description="Date and time of the event (ISO 8601)")
    duration_minutes: int = Field(description="Duration in minutes")
    participants: list[str] = Field(description="List of participants")


In [18]:
class Change(BaseModel):
    """Details for changing an existing event"""

    field: str = Field(description="Field to change")
    new_value: str = Field(description="New value for the field")

In [19]:
class ModifyEventDetails(BaseModel):
    """Details for modifying an existing event"""

    event_identifier: str = Field(
        description="Description to identify the existing event"
    )
    changes: list[Change] = Field(description="List of changes to make")
    participants_to_add: list[str] = Field(description="New participants to add")
    participants_to_remove: list[str] = Field(description="Participants to remove")

In [20]:
class CalendarResponse(BaseModel):
    """Final response format"""

    success: bool = Field(description="Whether the operation was successful")
    message: str = Field(description="User-friendly response message")
    calendar_link: Optional[str] = Field(description="Calendar link if applicable")

##### Step 2: Define the routing and processing functions


In [21]:
def route_calendar_request(user_input: str) -> CalendarRequestType:
    """Router LLM call to determine the type of calendar request"""
    logger.info("Routing calendar request")

    response = client.responses.parse(
    model="gpt-4o",
    input=user_input,
    instructions="Determine if this is a request to create a new calendar event or modify an existing one.",
    text_format=CalendarRequestType,
    )
    result = response.output[0].content[0].parsed
    
    logger.info(
        f"Request routed as: {result.request_type} with confidence: {result.confidence_score}"
    )
    return result

In [22]:
def handle_new_event(description: str) -> CalendarResponse:
    """Process a new event request"""
    logger.info("Processing new event request")

    today = datetime.now()
    date_context = f"Today is {today.strftime('%A, %B %d, %Y')}."

    # Get event details
    response = client.responses.parse(
    model="gpt-4o",
    input=description,
    instructions=f"{date_context} Extract details for creating a new calendar event.",
    text_format=NewEventDetails,
    )
    details = response.output[0].content[0].parsed

    logger.info(f"New event: {details.model_dump_json(indent=2)}")

    # Generate response
    return CalendarResponse(
        success=True,
        message=f"Created new event '{details.name}' for {details.date} with {', '.join(details.participants)}",
        calendar_link=f"calendar://new?event={details.name}",
    )

In [23]:
def handle_modify_event(description: str) -> CalendarResponse:
    """Process an event modification request"""
    logger.info("Processing event modification request")

    today = datetime.now()
    date_context = f"Today is {today.strftime('%A, %B %d, %Y')}."
    
    # Get modification details
    response = client.responses.parse(
    model="gpt-4o",
    input=description,
    instructions=f"{date_context} Extract details for modifying an existing calendar event.",
    text_format=ModifyEventDetails,
    )
    details = response.output[0].content[0].parsed

    logger.info(f"Modified event: {details.model_dump_json(indent=2)}")

    # Generate response
    return CalendarResponse(
        success=True,
        message=f"Modified event '{details.event_identifier}' with the requested changes",
        calendar_link=f"calendar://modify?event={details.event_identifier}",
    )

#### Router

- Classifies the request type (new/modify event)
- Provides confidence scoring
- Cleans and standardizes the input

In [24]:
def process_calendar_request(user_input: str) -> Optional[CalendarResponse]:
    """Main function implementing the routing workflow"""
    logger.info("Processing calendar request")

    # Route the request
    route_result = route_calendar_request(user_input)

    # Check confidence threshold
    if route_result.confidence_score < 0.7:
        logger.warning(f"Low confidence score: {route_result.confidence_score}")
        return None

    # Route to appropriate handler
    if route_result.request_type == "new_event":
        return handle_new_event(route_result.description)
    elif route_result.request_type == "modify_event":
        return handle_modify_event(route_result.description)
    else:
        logger.warning("Request type not supported")
        return None

##### Step 3: Test with new event

In [25]:
new_event_input = "Let's schedule a team meeting next Tuesday at 2pm with Alice and Bob"
result = process_calendar_request(new_event_input)
if result:
    print(f"Response: {result.message}")

2025-05-05 14:37:28 - INFO - Processing calendar request
2025-05-05 14:37:28 - INFO - Routing calendar request
2025-05-05 14:37:30 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-05-05 14:37:30 - INFO - Request routed as: new_event with confidence: 0.95
2025-05-05 14:37:30 - INFO - Processing new event request
2025-05-05 14:37:32 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-05-05 14:37:32 - INFO - New event: {
  "name": "Team Meeting",
  "date": "2025-05-13T14:00:00",
  "duration_minutes": 60,
  "participants": [
    "Alice",
    "Bob"
  ]
}


Response: Created new event 'Team Meeting' for 2025-05-13T14:00:00 with Alice, Bob


##### Step 4: Test with modify event

In [26]:
modify_event_input = (
    "Can you move the team meeting with Alice and Bob to Wednesday at 3pm instead?"
)
result = process_calendar_request(modify_event_input)
if result:
    print(f"Response: {result.message}")


2025-05-05 14:37:40 - INFO - Processing calendar request
2025-05-05 14:37:40 - INFO - Routing calendar request
2025-05-05 14:37:42 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-05-05 14:37:42 - INFO - Request routed as: modify_event with confidence: 0.95
2025-05-05 14:37:42 - INFO - Processing event modification request
2025-05-05 14:37:43 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-05-05 14:37:44 - INFO - Modified event: {
  "event_identifier": "Team meeting with Alice and Bob",
  "changes": [
    {
      "field": "Date and Time",
      "new_value": "Wednesday at 3pm"
    }
  ],
  "participants_to_add": [],
  "participants_to_remove": []
}


Response: Modified event 'Team meeting with Alice and Bob' with the requested changes


##### Step 5: Test with invalid request

In [27]:
invalid_input = "What's the weather like today?"
result = process_calendar_request(invalid_input)
if not result:
    print("Request not recognized as a calendar operation")

2025-05-05 14:38:01 - INFO - Processing calendar request
2025-05-05 14:38:01 - INFO - Routing calendar request
2025-05-05 14:38:02 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-05-05 14:38:02 - INFO - Request routed as: other with confidence: 0.95
2025-05-05 14:38:02 - WARNING - Request type not supported


Request not recognized as a calendar operation
